In [56]:
import argparse
import json

import gradio as gr
import requests


def fetch_hostname_for_vllm():
    # Execute the qstat-gpu command and store its output in a variable
    qstat_output = !qstat -lfu $USER
    # Find the line that contains 'Location'
    location_line = [line for line in qstat_output if 'Location' in line][0]
    # Extract the location value and append .pub.alcf.anl.gov
    location = location_line.split(":")[1].strip() + ".pub.alcf.anl.gov"
    return location



def http_bot(prompt, max_tokens=1024):
    headers = {"User-Agent": "vLLM Client"}
    pload = {
        "prompt": prompt,
        "stream": True,
        "max_tokens": max_tokens,
        "temperature": 0.0,
    }
    hostname=fetch_hostname_for_vllm()
    response = requests.post(f"http://{hostname}:8000/generate",
                             headers=headers,
                             json=pload,
                             stream=True)

    for chunk in response.iter_lines(chunk_size=8192,
                                     decode_unicode=False,
                                     delimiter=b"\0"):
        if chunk:
            data = json.loads(chunk.decode("utf-8"))
            output = data["text"][0]
            yield output

            
def build_demo():
    with gr.Blocks() as demo:
        gr.Markdown("# vLLM text completion demo\n")
        inputbox = gr.Textbox(label="Input",
                              placeholder="Enter text and press ENTER")
        max_tokens = gr.Number(label="Max Tokens", value=1024, minimum=1, maximum=10000)
        outputbox = gr.Textbox(label="Output",
                               placeholder="Generated result from the model")
        inputbox.submit(http_bot, [inputbox,max_tokens], [outputbox])
    return demo

demo = build_demo()
#demo.launch(inline=True,server_name="0.0.0.0")
demo.queue(concurrency_count=100).launch(server_name='0.0.0.0', debug=True, share=True)


Running on local URL:  http://0.0.0.0:7896
Running on public URL: https://e825aa692fa9fe7751.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 

In [54]:
import sys
import os
import re
import json
import requests
import time
import pandas as pd


def fetch_hostname_for_vllm():
    # Execute the qstat-gpu command and store its output in a variable
    qstat_output = !qstat -lfu $USER
    # Find the line that contains 'Location'
    location_line = [line for line in qstat_output if 'Location' in line][0]
    # Extract the location value and append .pub.alcf.anl.gov
    location = location_line.split(":")[1].strip() + ".pub.alcf.anl.gov"
    return location

def call_vllm_api_for_llama(prompt, max_tokens=1024):
    headers = {"User-Agent": "vLLM Client"}
    pload = {
        "prompt": prompt,
        #"stream": True,
        "max_tokens": max_tokens,
        "temperature": 0.0,
    }
    hostname=fetch_hostname_for_vllm()
    response = requests.post(f"http://{hostname}:8000/generate",
                             headers=headers,
                             json=pload)
    return response
    


def call_llama_code(argument='RAD51'):
    start_time = time.time()
    df = pd.read_csv('/home/atanikanti/LLM-service-api/balsam_service_ppi_llm_70B/open-ai-implementation/proteins.csv', header=None, names=['search_words'])
    with open("prompt_chat_gpt.output.txt","w") as f:

    #     openai.api_key = os.getenv("OPENAI_API_KEY")
    #     print("key",openai.api_key)
    #     model_engine = "gpt-3.5-turbo" 
    #     # This specifies which GPT model to use, as there are several models available, each with different capabilities and performance characteristics.

        # open the input file and read its contents
        depth = 30
        w_list =[] # work list of proteins not prompted yet
        s_list =[] # seen list of proteins we have already prompted


    #         argument = sys.argv[1]

        if (not argument):
            with open('prompts.txt', 'r') as ff:
                strings = ff.readlines()
                print("no arguments")
                for string in strings:
                    w_list.append(string.strip())
        else:
            w_list.append(argument.strip())            

        w_list = list(set(w_list))        
        print(w_list,file=f)
        leng = len(w_list)
        while depth > 0 and leng > 0:
            depth = depth - 1
            target = w_list[0]
            s_list.append(target)
            del w_list[0]
            print("------------------------------------------",file=f)
            print("Target -->", target,file=f)
            model_start_time = time.time()
            prompt = "You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with " + target
            response = call_vllm_api_for_llama(prompt)
#             response = openai.ChatCompletion.create(
#                 model='gpt-3.5-turbo',
#                 messages=[
#                     {"role": "system", "content": "You are a helpful assistant."},
#                     {"role": "user", "content": "We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with " + target},
#                 ])
            
            
            model_end_time = time.time()
            elapsed_model_time = model_end_time - model_start_time
            print('Total model execution time:', elapsed_model_time, 'seconds')                        
            result = ''
#           for choice in response:
            for output in response.iter_lines(decode_unicode=False,delimiter=b"\0"):
                data = json.loads(output.decode("utf-8"))
                choice = data["text"][0]
                print(f'The response of llama model for target {target} is {choice}')
                result += choice
                print(result, file=f)
                # Define the string to be searched
                search_string = result
                # Define an empty list to store the matches
                matches = []
                # Loop through the words in the DataFrame
                for word in df['search_words']:
                    # Check if the word is in the search string
                    pattern = r"\b{}\b".format(word)
                    match = re.search(pattern, search_string)
                    if match:
                        # If it is, append the word to the matches list
                        matches.append(word)
                    # Convert the matches list into a new DataFrame
                matches_u = list(set(matches))        
                matches_df = pd.DataFrame({'match_words': matches_u})
                # Print the new DataFrame
                matches_df = matches_df.drop_duplicates()
                for word in matches_df['match_words']:
                    if target != word:
                        print(target,"-->", word)
                        w_list.append(word)
                w_list = list(set(w_list)) # remove duplicates before removing previously seen
                for item in s_list:
                    if item in w_list:
                        w_list.remove(item)
                print(w_list, file=f)
                print("-----------------------------------------------",file=f)
                print(" ",file=f)
    # get the end time
    end_time = time.time()
    # get the execution time
    elapsed_time = end_time - start_time
    print('Total block execution time:', elapsed_time, 'seconds')                        


In [55]:
import os
import subprocess

arg = 'RAD51'

# Store the current working directory
os.chdir('/home/atanikanti/vllm_service/thetagpu')

# Check if directory exists, if not, create it
if not os.path.exists(f"DIR.{arg}"):
    os.mkdir(f"DIR.{arg}")

# Copy files into the directory
#os.system(f"cp prompt_chat_GPT.py DIR.{arg}")
#os.system(f"cp proteins.csv DIR.{arg}")

# Change to the directory
os.chdir(f"/home/atanikanti/vllm_service/thetagpu/DIR.{arg}")

# Loop and execute commands
for i in range(1, 11):
    # Ensure we're in the right directory
    print(i)
    print(arg)
    
    # Execute the python script and redirect output
    with open(f"{i}.output.txt", "w") as output_file:
        #subprocess.run(["python", "./prompt_chat_GPT.py", arg], stdout=output_file)
        call_llama_code(arg)
    
    # Print the number of lines for the created output file and the original file
    os.system(f"wc -l {i}.output.txt")
    os.system("wc -l prompt_chat_gpt.output.txt")
    
    # Rename the file
    os.rename("prompt_chat_gpt.output.txt", f"{i}.long.output.txt")

1
RAD51
Total model execution time: 28.90601682662964 seconds
The response of llama model for target RAD51 is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with RAD51?

RAD51 is a protein that plays a crucial role in the repair of DNA double-strand breaks. It is a key component of the homologous recombination pathway, which helps to maintain genome stability by repairing DNA damage through the exchange of genetic information with a template. RAD51 is a recombinase that is responsible for the recognition and binding of homologous DNA sequences, and it plays a central role in the repair of DNA double-strand breaks.

Certainly! Based on the documents I have been trained with, there are several proteins that have been identified as interacting with RAD51. These include:

1. RAD52: RAD52 is a protein that is closely related to RAD51 and also plays a 

ATRIP --> CDT1
ATRIP --> CDC6
ATRIP --> MCM7
ATRIP --> ORC2
ATRIP --> MCM5
ATRIP --> MCM6
ATRIP --> ORC1
ATRIP --> MCM3
ATRIP --> MCM2
Total model execution time: 25.787118911743164 seconds
The response of llama model for target CDT1 is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with CDT1?

CDT1 is a protein that helps regulate the cell cycle and is often overexpressed in cancer. It is a key regulator of the G1/S checkpoint and is involved in the regulation of DNA replication and cell proliferation.

I'm happy to help! Based on the information you provided, CDT1 is a protein that plays a crucial role in regulating the cell cycle and is often overexpressed in cancer. There are several proteins that have been identified as interacting with CDT1, including:

1. CDKN1A: CDKN1A is a protein that inhibits the activity of CDT1 and is involved in the

XRCC3 --> ATR
XRCC3 --> XRCC5
XRCC3 --> XRCC2
XRCC3 --> ATM
XRCC3 --> XRCC4
Total model execution time: 24.370876789093018 seconds
The response of llama model for target RAD51B is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with RAD51B?

RAD51B is a protein that plays a role in DNA repair and is associated with various cancer types.

I can certainly help you with that! Based on the documents I have been trained with, RAD51B has been reported to interact with several proteins that are involved in DNA repair and other cellular processes. Here are some examples:

1. RAD51: RAD51B is a paralog of RAD51, and the two proteins have been shown to interact and collaborate in DNA repair processes.
2. BRCA1: BRCA1 is a tumor suppressor protein that is involved in DNA repair and is associated with breast and ovarian cancer. RAD51B has been reported to int

CDK5RAP2 --> KAT2A
CDK5RAP2 --> CDK5
CDK5RAP2 --> CBX3
CDK5RAP2 --> BRCA1
Total model execution time: 26.61177635192871 seconds
The response of llama model for target CDC25A is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with CDC25A?

CDC25A is a protein that is involved in the regulation of the cell cycle and is often overexpressed in various types of cancer. It is a tyrosine phosphatase that dephosphorylates and activates key regulatory proteins, including cyclin-dependent kinases (CDKs) and retinoblastoma protein (Rb).

I can provide you with a list of proteins that have been reported to interact with CDC25A, based on the literature that I have been trained on. Here are some examples:

1. CDKs: CDC25A has been shown to interact with and dephosphorylate CDKs, such as CDK1, CDK2, and CDK4, which are key regulators of the cell cycle.
2. Rb: CD

ORC2 --> CDC6
ORC2 --> MCM2
ORC2 --> ORC1
Total model execution time: 29.79485774040222 seconds
The response of llama model for target CDK1 is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with CDK1?

  Sure! I can help you with that. CDK1 (Cyclin-dependent kinase 1) is a key regulator of the cell cycle and has been implicated in various cellular processes, including DNA damage response and repair. There are several proteins that have been reported to interact with CDK1. Here are some examples:

1. Cyclins: Cyclins are a family of proteins that bind to and activate CDKs, including CDK1. Cyclin B1, B2, and B3 have been shown to interact with CDK1 and play a role in regulating the G2/M checkpoint and progression through mitosis.
2. CDKN1A (Cyclin-dependent kinase inhibitor 1A): CDKN1A is a negative regulator of CDK1 and other CDKs. It binds to CDK

...

ATM --> RAD50
ATM --> ATR
ATM --> ATRIP
Total model execution time: 24.05156445503235 seconds
The response of llama model for target APC is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with APC?

APC is a protein that plays a critical role in cellular processes such as cell adhesion, migration, and signaling. It is a large protein composed of multiple domains, including an N-terminal domain, a central domain, and a C-terminal domain. The protein is known to interact with a variety of other proteins, including other adhesion molecules, signaling molecules, and cytoskeleton components.

One of the key functions of APC is to regulate the stability of cadherin-based adherens junctions, which are important for maintaining tissue architecture and cell-cell adhesion. APC has been shown to interact with cadherins and other adhesion molecules, such as b

PML --> SUV39H1
PML --> E2F1
PML --> CBX3
PML --> HDAC1
PML --> DNMT1
Total block execution time: 753.4923584461212 seconds
0 2.output.txt
647 prompt_chat_gpt.output.txt
3
RAD51
Total model execution time: 28.802106618881226 seconds
The response of llama model for target RAD51 is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with RAD51?

RAD51 is a protein that plays a crucial role in the repair of DNA double-strand breaks. It is a key component of the homologous recombination pathway, which helps to maintain genome stability by repairing DNA damage through the exchange of genetic information with a template. RAD51 is a recombinase that is responsible for the recognition and binding of homologous DNA sequences, and it plays a central role in the repair of DNA double-strand breaks.

Certainly! Based on the documents I have been trained with, ther

ATRIP --> CDT1
ATRIP --> CDC6
ATRIP --> MCM7
ATRIP --> ORC2
ATRIP --> MCM5
ATRIP --> MCM6
ATRIP --> ORC1
ATRIP --> MCM3
ATRIP --> MCM2
Total model execution time: 25.56744360923767 seconds
The response of llama model for target CDT1 is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with CDT1?

CDT1 is a protein that helps regulate the cell cycle and is often overexpressed in cancer. It is a key regulator of the G1/S checkpoint and is involved in the regulation of DNA replication and cell proliferation.

I'm happy to help! Based on the information you provided, CDT1 is a protein that plays a crucial role in regulating the cell cycle and is often overexpressed in cancer. There are several proteins that have been identified as interacting with CDT1, including:

1. CDKN1A: CDKN1A is a protein that inhibits the activity of CDT1 and is involved in the 

XRCC3 --> ATR
XRCC3 --> XRCC5
XRCC3 --> XRCC2
XRCC3 --> ATM
XRCC3 --> XRCC4
Total model execution time: 24.338238954544067 seconds
The response of llama model for target RAD51B is You are a helpful assistant. We are interested in protein interactions.  Based on the documents you have been trained with, can you provide any information on which proteins might interact with RAD51B?

RAD51B is a protein that plays a role in DNA repair and is associated with various cancer types.

I can certainly help you with that! Based on the documents I have been trained with, RAD51B has been reported to interact with several proteins that are involved in DNA repair and other cellular processes. Here are some examples:

1. RAD51: RAD51B is a paralog of RAD51, and the two proteins have been shown to interact and collaborate in DNA repair processes.
2. BRCA1: BRCA1 is a tumor suppressor protein that is involved in DNA repair and is associated with breast and ovarian cancer. RAD51B has been reported to int

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [4]:
"""Example Python client for vllm.entrypoints.api_server"""

import argparse
import json
from typing import Iterable, List

import requests


def clear_line(n: int = 1) -> None:
    LINE_UP = '\033[1A'
    LINE_CLEAR = '\x1b[2K'
    for _ in range(n):
        print(LINE_UP, end=LINE_CLEAR, flush=True)


def post_http_request(prompt: str,
                      api_url: str,
                      n: int = 1,
                      stream: bool = False,
                      temperature: float = 0.0,
                      max_tokens: int = 16) -> requests.Response:
    headers = {"User-Agent": "Test Client"}
    pload = {
        "prompt": prompt,
        "n": n,
        "use_beam_search": True,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stream": stream,
    }
    response = requests.post(api_url, headers=headers, json=pload, stream=True)
    return response


def get_streaming_response(response: requests.Response) -> Iterable[List[str]]:
    for chunk in response.iter_lines(chunk_size=8192,
                                     decode_unicode=False,
                                     delimiter=b"\0"):
        if chunk:
            data = json.loads(chunk.decode("utf-8"))
            output = data["text"]
            yield output


def get_response(response: requests.Response) -> List[str]:
    data = json.loads(response.content)
    output = data["text"]
    return output

hostname = fetch_hostname_for_vllm()

api_url = f"http://{hostname}:8000/generate"
n = 4
stream = True
temperature = 0.0
max_tokens = 16
prompt = "San Francisco is a"
response = post_http_request(prompt, api_url, n, stream, temperature, max_tokens)

if stream:
    num_printed_lines = 0
    for h in get_streaming_response(response):
        clear_line(num_printed_lines)
        num_printed_lines = 0
        for i, line in enumerate(h):
            num_printed_lines += 1
            print(f"Beam candidate {i}: {line!r}", flush=True)
else:
    output = get_response(response)
    for i, line in enumerate(output):
        print(f"Beam candidate {i}: {line!r}", flush=True)


Beam candidate 0: 'San Francisco is acity'
Beam candidate 1: 'San Francisco is agreat'
Beam candidate 2: 'San Francisco is abeautiful'
Beam candidate 3: 'San Francisco is avery'
Beam candidate 0: 'San Francisco is acity of'
Beam candidate 1: 'San Francisco is acity that'
Beam candidate 2: 'San Francisco is abeautiful city'
Beam candidate 3: 'San Francisco is agreat place'
Beam candidate 0: 'San Francisco is agreat place to'
Beam candidate 1: 'San Francisco is acity that is'
Beam candidate 2: 'San Francisco is abeautiful city,'
Beam candidate 3: 'San Francisco is abeautiful city with'
Beam candidate 0: 'San Francisco is agreat place to live'
Beam candidate 1: 'San Francisco is agreat place to visit'
Beam candidate 2: 'San Francisco is agreat place to be'
Beam candidate 3: 'San Francisco is abeautiful city, but'
Beam candidate 0: 'San Francisco is agreat place to live,'
Beam candidate 1: 'San Francisco is agreat place to live and'
Beam candidate 2: 'San Francisco is agreat place to visit